In [ ]:

bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
intron_conservation_3p = pd.read_csv('{}/exon_3p_upstream200_placental_mammal_conservation.csv'.format(bed_folder), 
                                     index_col=0)
intron_conservation_5p = pd.read_csv('{}/exon_5p_downstream200_placental_mammal_conservation.csv'.format(bed_folder), 
                                     index_col=0)

intron_conservations = {'Upstream of $3^\prime$': intron_conservation_3p, 
                        'Downstream of $5^\prime$': intron_conservation_5p}

for phenotype, phenotype_df in modalities_tidy.groupby('phenotype'):
    modalities = phenotype_df.set_index('event_name').modality
    modalities = modalities.drop(modalities.index.get_duplicates())
    
    
    dfs = []
    for intron_name, intron_conservation in intron_conservations.items():
        mean_conservation = intron_conservation.groupby(modalities).mean()
        mean_conservation.columns = mean_conservation.columns.astype(int) + 1
        mean_conservation.head()
        tidy = mean_conservation.unstack().reset_index()
        tidy = tidy.rename(columns={'level_0':'genome_direction_nt', 0: 'Conservation'})
        tidy['direction'] = intron_name
        dfs.append(tidy)
#         var_conservation = intron_conservation.groupby(modalities).var()
    modality_basewise_conservation = pd.concat(dfs, ignore_index=True)
    
    g = sns.FacetGrid(modality_basewise_conservation, hue='modality', hue_order=modality_order_constit,
                     palette=modality_colors_constit, 
                      col='direction', sharex=False,
                      col_order=['Upstream of $3^\prime$', 'Downstream of $5^\prime$'], size=3)
    g.map(plt.plot, 'genome_direction_nt', 'Conservation')
    for ax in g.axes.flat:
        ax.locator_params(nbins=4)
#         ax.locator_params('y', nbins=4)
        if ax.is_first_col():
            ax.set(ylabel='Mean PhastCons score\n(placental mammals)',
                   xticklabels=list(reversed([int(x) for x in ax.get_xticks()])))
        else:
            sns.despine(ax=ax, left=True)
            ax.yaxis.set_ticks_position('none')
    g.set(xlabel='Nucleotides from exon', title='', ylim=(0, 1))
    g.add_legend();
    g.savefig('{}/{}_intron_basewise_conservation.pdf'.format(folder, phenotype))